In [2]:
# Importar librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---------------------------
# 1. Descargar página web
# ---------------------------
url = "https://www.mercadolibre.com.mx"
pagina = requests.get(url)
print("Primeras 100 letras de la página:")
print(pagina.text[:100])

# ---------------------------
# 2. Hacer scraping de productos
# ---------------------------
url_busqueda = "https://listado.mercadolibre.com.mx/laptops"
pagina = requests.get(url_busqueda)
sopa = BeautifulSoup(pagina.text, 'html.parser')

productos = []
for item in sopa.find_all('li', class_='ui-search-layout__item')[:5]:  # Solo 5 productos
    nombre = item.find('h2').text
    precio = item.find('span', class_='andes-money-amount__fraction').text
    productos.append({'Producto': nombre, 'Precio': f"${precio} MXN"})

# Mostrar lista de productos
print("\nLista de laptops:")
df_productos = pd.DataFrame(productos)
print(df_productos)

# ---------------------------
# 3. Usar una API pública
# ---------------------------
url_api = "https://api.github.com/users/octocat/repos"
datos_api = requests.get(url_api).json()

repositorios = []
for repo in datos_api[:5]:  # Solo 5 repositorios
    repositorios.append({
        'Nombre': repo['name'],
        'Lenguaje': repo['language'] or "No especificado",
        'Estrellas': repo['stargazers_count']
    })

# Mostrar lista ordenada por estrellas
print("\nRepositorios de GitHub (ordenados por estrellas):")
df_repos = pd.DataFrame(repositorios)
df_repos = df_repos.sort_values('Estrellas', ascending=False)
print(df_repos)

ProxyError: HTTPSConnectionPool(host='www.mercadolibre.com.mx', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden')))

In [3]:
pip install requests beautifulsoup4 pandas fake-useragent

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/51/37/b3ea9cd5558ff4cb51957caca2193981c6b0ff30bd0d2630ac62505d99d0/fake_useragent-2.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.0 MB/s eta 0:00:00.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from fake_useragent import UserAgent

# Configurar User-Agent aleatorio
ua = UserAgent()

# ---------------------------
# 1. Descargar página web (con headers)
# ---------------------------
headers = {
    'User-Agent': ua.random,
    'Accept-Language': 'es-MX,es;q=0.9'
}

url = "https://www.mercadolibre.com.mx"
try:
    pagina = requests.get(url, headers=headers, timeout=10)
    pagina.raise_for_status()
    print("Primeras 100 letras de la página:")
    print(pagina.text[:100])
except Exception as e:
    print(f"Error al acceder a la página principal: {e}")

# ---------------------------
# 2. Hacer scraping de productos (con protección anti-bots)
# ---------------------------
url_busqueda = "https://listado.mercadolibre.com.mx/laptops"
try:
    # Cambiamos el User-Agent para cada solicitud
    headers['User-Agent'] = ua.random
    
    pagina = requests.get(url_busqueda, headers=headers, timeout=10)
    pagina.raise_for_status()
    
    # Verificar si no nos redirige a página de verificación
    if "validate" in pagina.url:
        raise Exception("El sitio nos redirigió a verificación anti-bots")
    
    sopa = BeautifulSoup(pagina.text, 'html.parser')
    
    # Selectores actualizados (pueden cambiar)
    productos = []
    for item in sopa.select('li.ui-search-layout__item:not(.ui-search-advertisement)')[:5]:
        try:
            nombre = item.find('h2', class_='ui-search-item__title').text.strip()
            precio = item.find('span', class_='andes-money-amount__fraction').text.strip()
            productos.append({'Producto': nombre, 'Precio': f"${precio} MXN"})
        except AttributeError as e:
            continue
    
    # Pausa para evitar bloqueos
    time.sleep(2)
    
    print("\nLista de laptops:")
    if productos:
        df_productos = pd.DataFrame(productos)
        print(df_productos)
    else:
        print("No se encontraron productos. Puede que los selectores hayan cambiado.")
        
except Exception as e:
    print(f"Error durante el scraping: {e}")

# ---------------------------
# 3. Usar una API pública (esta parte debería funcionar)
# ---------------------------
try:
    url_api = "https://api.github.com/users/octocat/repos"
    response = requests.get(url_api, timeout=10)
    response.raise_for_status()
    
    datos_api = response.json()
    
    repositorios = []
    for repo in datos_api[:5]:
        repositorios.append({
            'Nombre': repo['name'],
            'Lenguaje': repo['language'] or "No especificado",
            'Estrellas': repo['stargazers_count']
        })
    
    print("\nRepositorios de GitHub (ordenados por estrellas):")
    df_repos = pd.DataFrame(repositorios)
    df_repos = df_repos.sort_values('Estrellas', ascending=False)
    print(df_repos)
    
except Exception as e:
    print(f"Error al consumir la API: {e}")

ModuleNotFoundError: No module named 'fake_useragent'

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from fake_useragent import UserAgent

# 1. Configuración inicial
ua = UserAgent()
headers = {
    'User-Agent': ua.random,
    'Accept-Language': 'es-MX,es;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/'
}

# 2. Función para hacer solicitudes seguras
def hacer_solicitud(url):
    try:
        headers['User-Agent'] = ua.random
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        if response.status_code == 200:
            if "captcha" in response.url or "validate" in response.url:
                raise Exception("Redireccionado a verificación CAPTCHA")
            return response
        else:
            raise Exception(f"Error HTTP: {response.status_code}")
    except Exception as e:
        print(f"Error en la solicitud: {e}")
        return None

# 3. Prueba de conexión básica
print("=== PRUEBA DE CONEXIÓN ===")
url_prueba = "https://www.mercadolibre.com.mx"
response_prueba = hacer_solicitud(url_prueba)
if response_prueba:
    print("Conexión exitosa. Estado:", response_prueba.status_code)
    print("Encabezados recibidos:")
    for header, value in response_prueba.headers.items():
        print(f"{header}: {value}")

# 4. Scraping de productos (versión mejorada)
def scrapear_productos():
    print("\n=== SCRAPING DE PRODUCTOS ===")
    url_busqueda = "https://listado.mercadolibre.com.mx/laptops"
    response = hacer_solicitud(url_busqueda)
    
    if not response:
        print("No se pudo obtener la página de productos")
        return
    
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Selectores alternativos (por si cambian)
        posibles_selectores = [
            'li.ui-search-layout__item',
            'div.ui-search-result',
            'ol.ui-search-layout li'
        ]
        
        productos = []
        for selector in posibles_selectores:
            items = soup.select(selector)[:5]  # Limitar a 5 productos
            if items:
                break
        
        if not items:
            print("No se encontraron productos. Posibles causas:")
            print("- La estructura de la página ha cambiado")
            print("- El sitio está bloqueando el acceso")
            print("- Problema con los selectores CSS")
            return
        
        for item in items:
            try:
                nombre = item.find('h2', class_=lambda x: x and 'title' in x.lower()).text.strip()
                precio = item.find('span', class_=lambda x: x and 'price' in x.lower()).text.strip()
                link = item.find('a', href=True)['href']
                
                productos.append({
                    'Producto': nombre,
                    'Precio': precio,
                    'Enlace': link
                })
            except Exception as e:
                print(f"Error procesando un producto: {e}")
                continue
        
        # Mostrar resultados
        if productos:
            df = pd.DataFrame(productos)
            print("\nResultados encontrados:")
            print(df.to_string(index=False))
            
            # Guardar en CSV
            df.to_csv('productos_mercadolibre.csv', index=False)
            print("\nDatos guardados en 'productos_mercadolibre.csv'")
        else:
            print("No se extrajeron productos válidos")
            
    except Exception as e:
        print(f"Error durante el scraping: {e}")

# 5. Uso de API (versión mejorada)
def usar_api_github():
    print("\n=== CONSUMO DE API GITHUB ===")
    url_api = "https://api.github.com/users/octocat/repos"
    
    try:
        # Añadir headers específicos para API
        api_headers = {
            'User-Agent': 'Mozilla/5.0',
            'Accept': 'application/vnd.github.v3+json'
        }
        
        response = requests.get(url_api, headers=api_headers, timeout=10)
        response.raise_for_status()
        
        datos = response.json()
        
        repositorios = []
        for repo in datos[:5]:  # Limitar a 5 repositorios
            repositorios.append({
                'Nombre': repo.get('name', 'N/A'),
                'Lenguaje': repo.get('language', 'No especificado'),
                'Estrellas': repo.get('stargazers_count', 0),
                'Forks': repo.get('forks_count', 0),
                'URL': repo.get('html_url', '')
            })
        
        # Crear y mostrar DataFrame
        df = pd.DataFrame(repositorios)
        df = df.sort_values('Estrellas', ascending=False)
        
        print("\nRepositorios de GitHub:")
        print(df.to_string(index=False))
        
        # Guardar en JSON
        with open('repositorios_github.json', 'w', encoding='utf-8') as f:
            json.dump(repositorios, f, ensure_ascii=False, indent=2)
        print("\nDatos guardados en 'repositorios_github.json'")
        
    except Exception as e:
        print(f"Error al consumir la API: {e}")

# Ejecutar las funciones
if __name__ == "__main__":
    scrapear_productos()
    usar_api_github()

ModuleNotFoundError: No module named 'fake_useragent'